In [1]:
# Importing Libraries
import requests
import pandas as pd

In [2]:
class NewsApiClient:
    def __init__(self, api_key):
        self.api_key = api_key
        self.url = "https://newsapi.org/v2/everything?q=Indian%20market&apiKey={}".format(api_key)
        self.data = None
        
    def get_news(self):
        response = requests.get(self.url)
        data = response.json()
        news = data['articles']
        return news
    
    def store_data(self):
        news = self.get_news()
        df = pd.DataFrame(news)
        self.data = df.drop(columns=['source', 'author', 'url', 'urlToImage', 'publishedAt'])
        self.data.to_csv('NewsAPIClient.csv', index=False)
        return self.data

In [3]:
api_key = "726dad670180483fa1e7ba4381db5f69"
client = NewsApiClient(api_key)
df = client.store_data()

In [4]:
df.columns

Index(['title', 'description', 'content'], dtype='object')

In [5]:
# Text analyzer 1

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import sqlite3

In [16]:

class IndianMarketSentimentAnalyzer:
    def __init__(self, data):
        self.data = data
        self.text = None
        self.sentiment_analyzer = SentimentIntensityAnalyzer()
        self.conn = sqlite3.connect("Indian_Market_Sentiment.db")
        self.cursor = self.conn.cursor()
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS sentiment (
                sentiment text,
                probability text,
                text text
            )
        """)
    
    def analyze_sentiment(self):
        probability = self.sentiment_analyzer.polarity_scores(self.text)["compound"]
        return probability
    
    def interpret_sentiment(self):
        probability = self.analyze_sentiment()
        if probability > 0.05:
            return "Positive", probability
        elif probability < -0.05:
            return "Negative", probability
        else:
            return "Neutral", probability
    
    def add_data(self):
        for self.text in self.data['description']:
            sentiment, probability = self.interpret_sentiment()
            self.cursor.execute("""
                INSERT INTO sentiment (sentiment, probability, text)
                VALUES (?, ?, ?)
            """, (sentiment, probability, self.text))
            self.conn.commit()
            print(sentiment, probability, self.text)
        self.conn.close()

In [17]:
sentiment_analyzer = IndianMarketSentimentAnalyzer(df)
sentiment_analyzer.add_data()

Positive 0.296 Indian shares are set to open higher on Wednesday ahead of the Union budget, as investors look for major triggers for the near-term trajectory of the market.
Positive 0.7184 Indian gold futures hit a record high on Friday, tracking gains in overseas market, but the price rise dampened demand in the world's second biggest consumer of the precious metal, dealers said.
Positive 0.4215 The Indian rupee strengthened on Wednesday, helped by an uptick in most Asian emerging market peers, while traders noted there were some dollar inflows.
Neutral 0.0 OnePlus has some big plans for the Indian market, as many products are listed to launch next month, including the OnePlus 11 and the OnePlus Pad.
Neutral 0.0 Indian biopharma company Biocon <a href="https://www.reuters.com/companies/BION.NS" target="_blank">(BION.NS)</a> sold a 9.96% stake in its unit and contract research firm Syngene International through open market sale, it said in an exchange filing on Wednes…
Neutral 0.0 The 

In [18]:
conn = sqlite3.connect("Indian_Market_Sentiment.db")
cursor = conn.cursor()

data = cursor.execute('select * from sentiment')

In [19]:
data.fetchall()

[('Positive',
  '0.296',
  "BENGALURU, Feb 1 (Reuters) - Indian shares are set to open higher on Wednesday ahead of the Union budget, as investors look for major triggers for the near-term trajectory of the market.\r\nIndia's NSE… [+2075 chars]"),
 ('Neutral',
  '-0.0129',
  "MUMBAI, Jan 13 (Reuters) - Indian gold futures hit a record high on Friday, tracking gains in overseas market, but the price rise dampened demand in the world's second biggest consumer of the preciou… [+295 chars]"),
 ('Positive',
  '0.4215',
  'MUMBAI, Jan 18 (Reuters) - The Indian rupee strengthened on Wednesday, helped by an uptick in most Asian emerging market peers, while traders noted there were some dollar inflows.\r\nThe rupee finished… [+1476 chars]'),
 ('Negative',
  '-0.34',
  '<ul><li>OnePlus Pa renders leak online, showing off a unibody chassis and an unusual camera design.</li><li>It is placed in the middle of the back panel and features a large circular module.</li><li>… [+3027 chars]'),
 ('Neutral'